In [2]:
import numpy as np 
import pandas as pd
import os
import joblib
import cv2

from keras.layers import Dense, Flatten, MaxPooling2D, Dropout
from keras.models import Model
from keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

## Preprocessing

In [5]:
path = 'UTKFace'
files = os.listdir(path)
size = len(files)

In [6]:
size

23708

In [7]:
images = []
ages = []
genders = []
for i,file in enumerate(files):
  if i%500 == 0:

    print(i)
  ################
  image = cv2.imread(path+'/'+file)
  # image = cv2.resize(image,dsize=(64,64))
  # image = image.reshape((image.shape[0],image.shape[1],1))
  images.append(image)
  split_var = file.split('_')
  ages.append(split_var[0])
  genders.append(int(split_var[1]))

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500


In [8]:
resized = []
for im in images:
  resized.append(cv2.resize(im,(224,224)))

images = np.asarray(resized)

## free memory by deleting variable
del resized

In [12]:
x_train_gender, x_test_gender, y_train_gender, y_test_gender = train_test_split(images, genders, test_size=0.2,shuffle  = True,stratify=genders)
y_train_gender = np.asarray(y_train_gender)
y_test_gender = np.asarray(y_test_gender)

In [13]:
ages_array = np.asarray(ages,dtype=np.int)
scaler = MinMaxScaler()
ages_scaled = scaler.fit_transform(ages_array.reshape((-1,1)))
x_train_age, x_test_age, y_train_age, y_test_age = train_test_split(images, ages_scaled, test_size=0.2,shuffle  = True)

In [14]:
## free memory by deleting variable
del images

## Model for Gender Prediction

In [21]:
baseModel = VGG16(input_shape=(224,224,3), weights='imagenet', include_top=False)


for layer in baseModel.layers:
  layer.trainable = False

headModel = baseModel.output
headModel = MaxPooling2D()(headModel)
headModel = Flatten()(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.2)(headModel)
headModel = Dense(1, activation='sigmoid')(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

In [22]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [23]:
h = model.fit(x_train_gender,y_train_gender,validation_data=(x_test_gender,y_test_gender),epochs = 5, batch_size=32,shuffle = True)

Epoch 1/5
593/593 [==============================] - 97s 161ms/step - loss: 0.9088 - accuracy: 0.7869 - val_loss: 0.3105 - val_accuracy: 0.8648
Epoch 2/5
593/593 [==============================] - 94s 159ms/step - loss: 0.2898 - accuracy: 0.8708 - val_loss: 0.2710 - val_accuracy: 0.8768
Epoch 3/5
593/593 [==============================] - 94s 159ms/step - loss: 0.2426 - accuracy: 0.8928 - val_loss: 0.2877 - val_accuracy: 0.8806
Epoch 4/5
593/593 [==============================] - 94s 159ms/step - loss: 0.2141 - accuracy: 0.9037 - val_loss: 0.2891 - val_accuracy: 0.8754
Epoch 5/5
593/593 [==============================] - 94s 159ms/step - loss: 0.1980 - accuracy: 0.9094 - val_loss: 0.2871 - val_accuracy: 0.8781


In [24]:
model_json = model.to_json()
with open("gender_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("gender_model.h5")

## Model for Age Prediction

In [26]:
baseModel = VGG16(input_shape=(224,224,3), weights='imagenet', include_top=False)


for layer in baseModel.layers:
  layer.trainable = False

headModel = baseModel.output
headModel = MaxPooling2D()(headModel)
headModel = Flatten()(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.2)(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.2)(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.2)(headModel)
headModel = Dense(1, activation='linear')(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)


58900480/58889256 [==============================] - 0s 0us/step


In [27]:
model.compile(optimizer='adam',loss='mse',metrics=['mse','mae'])

In [28]:
h = model.fit(x_train_age,y_train_age,validation_data=(x_test_age,y_test_age),epochs = 5, batch_size=32,shuffle = True)

Epoch 1/20
593/593 [==============================] - 142s 158ms/step - loss: 1.8862 - mse: 1.8862 - mae: 0.6123 - val_loss: 0.0223 - val_mse: 0.0223 - val_mae: 0.1127
Epoch 2/20
593/593 [==============================] - 91s 153ms/step - loss: 0.0307 - mse: 0.0307 - mae: 0.1336 - val_loss: 0.0157 - val_mse: 0.0157 - val_mae: 0.0945
Epoch 3/20
593/593 [==============================] - 93s 157ms/step - loss: 0.0183 - mse: 0.0183 - mae: 0.1031 - val_loss: 0.0157 - val_mse: 0.0157 - val_mae: 0.0941
Epoch 4/20
593/593 [==============================] - 95s 160ms/step - loss: 0.0168 - mse: 0.0168 - mae: 0.0983 - val_loss: 0.0121 - val_mse: 0.0121 - val_mae: 0.0814
Epoch 5/20
593/593 [==============================] - 96s 162ms/step - loss: 0.0121 - mse: 0.0121 - mae: 0.0825 - val_loss: 0.0105 - val_mse: 0.0105 - val_mae: 0.0785
Epoch 6/20
593/593 [==============================] - 96s 163ms/step - loss: 0.0100 - mse: 0.0100 - mae: 0.0757 - val_loss: 0.0103 - val_mse: 0.0103 - val_mae: 0.07

In [41]:
model_json = model.to_json()
with open("age_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("age_model.h5")

In [ ]:
import joblib
scaler = joblib.load('scaler.pkl')